# Estimaciones JIRA (Story Points vs Horas reales)
Este cuaderno ejecuta el análisis a partir de un CSV exportado de JIRA.

**Qué genera** (en una carpeta de salida):
- `tabla_estimaciones.html`
- `tabla_estimaciones.png` (imagen equivalente a la tabla)
- `estimacion_vs_real.png`
- `tareas_dentro_fuera.png`
- `estado_quesito.png`

> Solo tienes que **subir el CSV** cuando se te pida y ejecutar las celdas en orden.


In [12]:
# Dependencias (Colab suele traerlas ya instaladas, pero lo dejamos por si acaso)
import sys, subprocess

def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Descomenta si tu entorno no trae estas libs (normalmente Colab sí)
# pip_install("pandas")
# pip_install("numpy")
# pip_install("matplotlib")

import re
import logging
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

logging.basicConfig(level=logging.INFO, format="%(levelname)s %(message)s")
logger = logging.getLogger("estimaciones")


## 1) Sube el CSV exportado de JIRA
Pulsa ▶ en la celda y selecciona el fichero CSV.

In [ ]:
from google.colab import files

uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No se subió ningún fichero. Vuelve a ejecutar y selecciona un CSV.")

# Nos quedamos con el primer fichero subido
csv_name = next(iter(uploaded.keys()))
csv_path = Path(csv_name)

print("CSV cargado:", csv_path)


## 2) Código del analizador
Esta celda define la clase `EstimacionesAnalyzer` y las funciones auxiliares para calcular la carpeta de salida por sprint.

In [ ]:
@dataclass
class EstimacionesAnalyzer:
    """Analiza un CSV de JIRA con Story Points y Tiempo empleado."""
    csv_path: str | Path
    day_hours: float = 8.0  # horas por día al parsear "1d 2h 30m"

    # Mapa SP -> etiqueta (texto) para mostrar en tablas
    sp_to_time: Dict[int, str] = field(default_factory=lambda: {
        1:  "1h",
        3:  "2-4h",
        5:  "6-8h (1 día)",
        8:  "15-25h (2-3 días)",
        13: "30-40h (4-5 días)",
        21: "1 sem y media",
        34: "2 sem",
        55: "1 sprint (3 sem)",
        89: "+1 sprint (separar)",
    })

    # Mapa SP -> (min_horas, max_horas) para cálculo
    sp_to_range: Dict[int, Tuple[float, float]] = field(default_factory=lambda: {
        1:  (0, 2),
        3:  (2, 5),
        5:  (5, 10),
        8:  (10, 25),
        13: (25, 45),
        21: (45, 70),     # ~1,5 semanas
        34: (70, 100),    # ~2 semanas
        55: (100, 130),   # ~1 sprint (3 semanas)
    })

    # Columnas esperadas en el CSV (después de renombrar)
    col_story_points: str = "Puntos de historia"
    col_time_spent: str = "Tiempo empleado"
    col_summary: str = "Resumen"

    df: pd.DataFrame | None = None
    df_total: pd.DataFrame | None = None
    df_sorted: pd.DataFrame | None = None

    # -----------------------
    # Carga y preprocesado
    # -----------------------
    def load(self) -> "EstimacionesAnalyzer":
        """Carga el CSV y hace limpieza básica."""
        self.df = pd.read_csv(self.csv_path)
        self._rename_columns()
        self._drop_empty_columns()
        self._drop_unused_columns()
        return self

    def _rename_columns(self) -> None:
        if self.df is None:
            raise RuntimeError("Primero llama a load().")

        rename_map = {
            "Campo personalizado (Story Points)": self.col_story_points,
            "Σ Tiempo empleado": self.col_time_spent,
        }
        self.df = self.df.rename(columns={k: v for k, v in rename_map.items() if k in self.df.columns})

    def _drop_empty_columns(self) -> None:
        if self.df is None:
            raise RuntimeError("Primero llama a load().")
        self.df = self.df.dropna(axis=1, how="all")

    def _drop_unused_columns(self) -> None:
        """Elimina columnas de JIRA típicamente irrelevantes (si existen)."""
        if self.df is None:
            raise RuntimeError("Primero llama a load().")
        cols_to_drop = ["ID de la persona asignada", "Persona asignada", "Principal", "Clave principal"]
        existing = [c for c in cols_to_drop if c in self.df.columns]
        if existing:
            self.df = self.df.drop(columns=existing)

    # -----------------------
    # Parsing de tiempo
    # -----------------------
    def parse_time_spent(self) -> "EstimacionesAnalyzer":
        """Crea la columna 'Horas dedicadas' a partir de Tiempo empleado."""
        self._require_df()

        if self.col_time_spent not in self.df.columns:
            raise KeyError(f"No existe la columna '{self.col_time_spent}' en el CSV.")

        self.df["Horas dedicadas"] = self.df[self.col_time_spent].apply(self._parse_time_spent_value)
        return self

    def _parse_time_spent_value(self, x) -> float:
        """
        Convierte:
          - numérico (asumido segundos) -> horas
          - string float ("3.5") -> horas
          - "1d 2h 30m" -> horas (días * day_hours + horas + minutos/60)
        """
        if pd.api.types.is_number(x):
            return float(x) / 3600.0

        if pd.isna(x):
            return np.nan

        s = str(x).strip()

        try:
            return float(s)
        except ValueError:
            pass

        pattern = r"(?:(\d+)\s*d)?\s*(?:(\d+)\s*h)?\s*(?:(\d+)\s*m)?"
        m = re.fullmatch(pattern, s)
        if not m:
            return np.nan

        days = int(m.group(1)) if m.group(1) else 0
        hours = int(m.group(2)) if m.group(2) else 0
        minutes = int(m.group(3)) if m.group(3) else 0

        return days * self.day_hours + hours + minutes / 60.0

    # -----------------------
    # Estimaciones por Story Points
    # -----------------------
    def add_story_points(self) -> "EstimacionesAnalyzer":
        """Normaliza story points y añade columnas de rango/etiqueta."""
        self._require_df()

        if self.col_story_points not in self.df.columns:
            raise KeyError(f"No existe la columna '{self.col_story_points}' en el CSV.")

        self.df["story_points_int"] = self.df[self.col_story_points].astype("Int64")
        self.df["Horas estimadas"] = self.df["story_points_int"].map(self.sp_to_time)
        self.df[["horas_est_min", "horas_est_max"]] = self.df[self.col_story_points].apply(self._sp_to_min_max)
        return self

    def _sp_to_min_max(self, sp) -> pd.Series:
        if pd.isna(sp):
            return pd.Series([np.nan, np.nan])
        try:
            sp_int = int(sp)
        except (TypeError, ValueError):
            return pd.Series([np.nan, np.nan])

        lo, hi = self.sp_to_range.get(sp_int, (np.nan, np.nan))
        return pd.Series([lo, hi])

    def compute_within_estimate(self) -> "EstimacionesAnalyzer":
        """Añade la columna 'Bien estimado?'."""
        self._require_df()

        def dentro_estimado(row) -> Optional[bool]:
            sp = row.get("story_points_int", pd.NA)
            horas = row.get("Horas dedicadas", np.nan)
            if pd.isna(sp) or pd.isna(horas):
                return np.nan
            rango = self.sp_to_range.get(int(sp)) if pd.notna(sp) else None
            if rango is None:
                return np.nan
            h_min, h_max = rango
            return bool(h_min <= horas <= h_max)

        self.df["Bien estimado?"] = self.df.apply(dentro_estimado, axis=1)
        return self

    def compute_error_percent(self) -> "EstimacionesAnalyzer":
        """Añade la columna 'error (%)' (0 dentro del rango)."""
        self._require_df()

        def error_porcentual(row) -> float:
            real = row.get("Horas dedicadas", np.nan)
            lo = row.get("horas_est_min", np.nan)
            hi = row.get("horas_est_max", np.nan)

            if pd.isna(real) or pd.isna(lo) or pd.isna(hi) or real == 0:
                return np.nan

            if lo <= real <= hi:
                return 0.0

            nearest = lo if abs(real - lo) <= abs(real - hi) else hi
            return (real - nearest) / real * 100.0

        self.df["error (%)"] = self.df.apply(error_porcentual, axis=1)
        return self

    # -----------------------
    # Totales y ordenación
    # -----------------------
    def add_total_row(self) -> "EstimacionesAnalyzer":
        """Crea df_total con una fila final de sumas en columnas numéricas."""
        self._require_df()

        num_cols = self.df.select_dtypes(include="number").columns
        total_row = {col: "" for col in self.df.columns}

        if self.col_summary in self.df.columns:
            total_row[self.col_summary] = "Total"
        else:
            total_row[self.df.columns[0]] = "Total"

        for col in num_cols:
            total_row[col] = pd.to_numeric(self.df[col], errors="coerce").sum()

        self.df_total = pd.concat([self.df, pd.DataFrame([total_row])], ignore_index=True)
        return self

    def sort(self, by: str = "Horas dedicadas", ascending: bool = True) -> "EstimacionesAnalyzer":
        """Ordena el dataframe (usa df_total si existe)."""
        base = self.df_total if self.df_total is not None else self.df
        if base is None:
            raise RuntimeError("No hay datos. Ejecuta load().")
        if by not in base.columns:
            raise KeyError(f"No existe la columna '{by}' para ordenar.")

        self.df_sorted = base.sort_values(by=by, ascending=ascending)
        return self

    # -----------------------
    # Estilos y exportación
    # -----------------------
    @staticmethod
    def _color_bien_estimado_bool(orig) -> str:
        if pd.isna(orig):
            return "background-color: yellow; color: black; font-weight: bold"
        if orig is True:
            return "background-color: lightgreen; color: black; font-weight: bold"
        if orig is False:
            return "background-color: lightcoral; color: black; font-weight: bold"
        return ""

    @staticmethod
    def _color_horas_estimadas(val) -> str:
        if pd.isna(val):
            return "background-color: lightcoral; color: black; font-weight: bold"
        return ""

    def _display_cols(self, base: pd.DataFrame) -> list[str]:
        return [c for c in [self.col_summary, self.col_story_points, "Horas dedicadas",
                            "Horas estimadas", "Bien estimado?", "error (%)"] if c in base.columns]

    def display_df(self) -> pd.DataFrame:
        """DF exacto que se pinta en el HTML, con formato listo para mostrar/guardar."""
        base = self.df_sorted if self.df_sorted is not None else (
            self.df_total if self.df_total is not None else self.df
        )
        if base is None:
            raise RuntimeError("No hay datos. Ejecuta el pipeline primero.")

        cols = self._display_cols(base)
        df_show = base[cols].copy()

        numeric_cols = df_show.select_dtypes(include="number").columns.tolist()
        for c in numeric_cols:
            if c == self.col_story_points:
                df_show[c] = pd.to_numeric(df_show[c], errors="coerce").round(0).astype("Int64")
            else:
                df_show[c] = pd.to_numeric(df_show[c], errors="coerce").round(1)

        return df_show

    def styled_table(self) -> "pd.io.formats.style.Styler":
        df_show = self.display_df()

        bien_orig = df_show["Bien estimado?"].copy() if "Bien estimado?" in df_show.columns else None

        if "Bien estimado?" in df_show.columns:
            df_show["Bien estimado?"] = ""

        fmt = {c: "{:.1f}" for c in df_show.select_dtypes(include="number").columns}
        if self.col_story_points in fmt:
            fmt[self.col_story_points] = "{:.0f}"

        styler = df_show.style.format(fmt, na_rep="")

        if bien_orig is not None:
            styler = styler.apply(
                lambda col: [self._color_bien_estimado_bool(v) for v in bien_orig],
                subset=["Bien estimado?"],
            )

        if "Horas estimadas" in df_show.columns:
            styler = styler.map(self._color_horas_estimadas, subset=["Horas estimadas"])

        return styler

    def save_table_image(self, out_png: str | Path) -> None:
        """
        Exporta una imagen PNG "equivalente" a la tabla HTML (misma info + colores).
        No depende de navegador/selenium.
        """
        base = self.df_sorted if self.df_sorted is not None else (self.df_total if self.df_total is not None else self.df)
        if base is None:
            raise RuntimeError("No hay datos. Ejecuta el pipeline primero.")

        cols = [c for c in [self.col_summary, self.col_story_points, "Horas dedicadas",
                            "Horas estimadas", "Bien estimado?", "error (%)"] if c in base.columns]
        df_img = base[cols].copy()
        bien_orig = df_img["Bien estimado?"].copy() if "Bien estimado?" in df_img.columns else None

        if "Bien estimado?" in df_img.columns:
            df_img["Bien estimado?"] = ""

        if "Horas dedicadas" in df_img.columns:
            df_img["Horas dedicadas"] = pd.to_numeric(df_img["Horas dedicadas"], errors="coerce").round(1)
        if "error (%)" in df_img.columns:
            df_img["error (%)"] = pd.to_numeric(df_img["error (%)"], errors="coerce").round(1)

        nrows, ncols = df_img.shape
        fig_w = max(10, ncols * 2.2)
        fig_h = max(2.5, nrows * 0.45)

        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis("off")

        table = ax.table(
            cellText=df_img.astype(str).values,
            colLabels=df_img.columns.tolist(),
            cellLoc="left",
            loc="center",
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 1.2)

        for c in range(ncols):
            cell = table[(0, c)]
            cell.set_text_props(weight="bold")

        col_idx = {name: i for i, name in enumerate(df_img.columns.tolist())}
        bien_col = col_idx.get("Bien estimado?")
        horas_est_col = col_idx.get("Horas estimadas")

        for r in range(1, nrows + 1):
            if bien_col is not None and bien_orig is not None:
                orig = bien_orig.iloc[r - 1]
                cell = table[(r, bien_col)]

                if pd.isna(orig):
                    cell.set_facecolor("yellow")
                    cell.set_text_props(weight="bold")
                elif orig is True:
                    cell.set_facecolor("lightgreen")
                    cell.set_text_props(weight="bold")
                elif orig is False:
                    cell.set_facecolor("lightcoral")
                    cell.set_text_props(weight="bold")

            if horas_est_col is not None:
                orig_he = base[cols].iloc[r - 1]["Horas estimadas"]
                cell = table[(r, horas_est_col)]
                if pd.isna(orig_he) or orig_he == "":
                    cell.set_facecolor("lightcoral")
                    cell.set_text_props(weight="bold")

        out_png = Path(out_png)
        out_png.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(out_png, bbox_inches="tight", dpi=200)
        plt.close(fig)

    def save_outputs(self, out_dir: str | Path) -> "EstimacionesAnalyzer":
        """
        Guarda:
          - tabla_estimaciones.html
          - tabla_estimaciones.png
          - (los plots se guardan desde run_all)

        IMPORTANTE: no se generan CSVs.
        """
        self._require_df()
        out = Path(out_dir)
        out.mkdir(parents=True, exist_ok=True)

        html = self.styled_table().to_html()
        (out / "tabla_estimaciones.html").write_text(html, encoding="utf-8")

        self.save_table_image(out / "tabla_estimaciones.png")

        logger.info("Outputs guardados en: %s", out)
        return self

    # -----------------------
    # Gráficos
    # -----------------------
    def plot_est_vs_real(self, show: bool = True, save_path: str | Path | None = None) -> None:
        """Bar chart: horas estimadas (punto medio) vs horas reales (media) por story point."""
        self._require_df()

        ranges_df = (
            pd.DataFrame.from_dict(self.sp_to_range, orient="index", columns=["h_min", "h_max"])
            .assign(h_mid=lambda d: (d["h_min"] + d["h_max"]) / 2)
        )
        mean_hours = self.df.groupby("story_points_int")["Horas dedicadas"].mean()
        merged = ranges_df.join(mean_hours).rename(columns={"Horas dedicadas": "h_real"}).dropna()

        x = np.arange(len(merged))
        plt.figure()
        plt.bar(x - 0.15, merged["h_mid"], width=0.3, label="Horas estimadas (medio)")
        plt.bar(x + 0.15, merged["h_real"], width=0.3, label="Horas reales (media)")
        plt.xticks(x, merged.index)
        plt.xlabel("Story points")
        plt.ylabel("Horas")
        plt.title("Estimación vs realidad por story point")
        plt.legend()
        plt.grid(axis="y")

        if save_path:
            plt.savefig(save_path, bbox_inches="tight")
        if show:
            plt.show()
        else:
            plt.close()

    def plot_within_out_counts(self, show: bool = True, save_path: str | Path | None = None) -> None:
        """Bar chart: conteo de tareas dentro/fuera de estimación (solo enteros)."""
        self._require_df()

        base = self.df_sorted if self.df_sorted is not None else (self.df_total if self.df_total is not None else self.df)
        if base is None:
            raise RuntimeError("No hay datos. Ejecuta el pipeline primero.")
        if "Bien estimado?" not in base.columns:
            raise RuntimeError("No existe 'Bien estimado?'. Ejecuta compute_within_estimate().")

        plot_df = base.copy()
        if self.col_summary in plot_df.columns:
            plot_df = plot_df[plot_df[self.col_summary] != "Total"]

        counts_raw = plot_df["Bien estimado?"].value_counts(dropna=False)

        def _label(v):
            if pd.isna(v):
                return "Sin dato"
            if v is True:
                return "Dentro"
            if v is False:
                return "Fuera"
            return str(v)

        counts = counts_raw.copy()
        counts.index = [_label(v) for v in counts.index]
        counts = counts.groupby(level=0).sum()

        order = ["Dentro", "Fuera", "Sin dato"]
        counts = counts.reindex(order).fillna(0).astype(int)

        colors = ["green", "red", "yellow"]

        fig, ax = plt.subplots()
        counts.plot(kind="bar", ax=ax, color=colors)
        ax.set_title("Tareas dentro / fuera de la estimación")
        ax.set_ylabel("Número de tareas")
        ax.grid(axis="y")
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        for p in ax.patches:
            h = int(round(p.get_height()))
            ax.annotate(
                f"{h}",
                (p.get_x() + p.get_width() / 2, p.get_height()),
                ha="center",
                va="bottom",
                fontsize=10,
            )

        plt.xticks(rotation=0)

        if save_path:
            plt.savefig(save_path, bbox_inches="tight")
        if show:
            plt.show()
        else:
            plt.close(fig)

    def plot_estado_pie(
        self,
        show: bool = True,
        save_path: str | Path | None = None,
        estado_col: str = "Estado",
        min_pct_label: float = 3.0,
    ) -> None:
        self._require_df()

        if estado_col not in self.df.columns:
            raise KeyError(f"No existe la columna '{estado_col}' en el CSV.")

        s = self.df[estado_col].fillna("Sin dato").astype(str).str.strip()
        counts = s.value_counts()

        desired_order = ["Cerrado", "Validando", "En curso", "Bloqueado", "Abierto", "Sin dato"]
        counts = counts.reindex(desired_order + [x for x in counts.index if x not in desired_order]).dropna()

        labels = counts.index.tolist()
        values = counts.values.astype(int).tolist()
        total = int(counts.sum())

        palette = {
            "Cerrado":   "#2E7D32",  # verde
            "Abierto":   "#0105FD",  # azul (no hecho)
            "En curso":  "#C8E6C9",  # verde muy claro
            "Validando": "#FB8C00",  # naranja
            "Bloqueado": "#D32F2F",  # rojo
            "Sin dato":  "#757575",  # gris
        }

        fallback_cycle = iter(plt.cm.tab20.colors)
        colors = []
        for lab in labels:
            colors.append(palette.get(lab, next(fallback_cycle)))

        explode = []
        for v in values:
            pct = 100.0 * v / total if total else 0.0
            explode.append(0.06 if pct < min_pct_label else 0.0)

        def _autopct(pct: float) -> str:
            return f"{pct:.1f}%" if pct >= min_pct_label else ""

        fig, ax = plt.subplots(figsize=(10, 6))
        wedges, *_ = ax.pie(
            values,
            labels=None,
            colors=colors,
            explode=explode,
            autopct=_autopct,
            startangle=90,
            pctdistance=0.70,
            wedgeprops={"linewidth": 1, "edgecolor": "white"},
            textprops={"fontsize": 11},
        )

        legend_labels = [
            f"{lab} — {v} ({(100.0*v/total if total else 0.0):.1f}%)"
            for lab, v in zip(labels, values)
        ]
        ax.legend(
            wedges,
            legend_labels,
            title="Estado",
            loc="center left",
            bbox_to_anchor=(1.02, 0.5),
            frameon=False,
        )

        ax.set_title("Distribución por estado")
        ax.axis("equal")
        fig.tight_layout()

        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            fig.savefig(save_path, bbox_inches="tight", dpi=300)

        if show:
            plt.show()
        else:
            plt.close(fig)

    # -----------------------
    # Pipeline completo
    # -----------------------
    def run_all(self, out_dir: str | Path | None = None) -> "EstimacionesAnalyzer":
        """Ejecuta el pipeline completo en el orden del notebook."""
        (
            self.load()
            .parse_time_spent()
            .add_story_points()
            .compute_within_estimate()
            .compute_error_percent()
            .add_total_row()
            .sort(by="Horas dedicadas", ascending=True)
        )

        if out_dir:
            out = Path(out_dir)
            out.mkdir(parents=True, exist_ok=True)

            self.save_outputs(out_dir)

            self.plot_est_vs_real(show=False, save_path=out / "estimacion_vs_real.png")
            self.plot_within_out_counts(show=False, save_path=out / "tareas_dentro_fuera.png")
            self.plot_estado_pie(show=False, save_path=out / "estado_quesito.png")

        return self

    # -----------------------
    # Helpers
    # -----------------------
    def _require_df(self) -> None:
        if self.df is None:
            raise RuntimeError("No hay dataframe cargado. Llama a load() primero.")


# --- Helpers para construir la ruta de salida por sprint (igual que tu script) ---

SPRINT_OVERRIDES: Dict[int, Tuple[int, int, int]] = {
    4825: (2025, 4, 2),
    4826: (2025, 4, 3),
    4827: (2025, 4, 4),
    4828: (2026, 1, 1),
}

BASE_SPRINT_NUM = 4825
BASE_YEAR, BASE_QUARTER, BASE_SPRINT_IN_QUARTER = (2025, 4, 2)

SPRINTS_PER_QUARTER = 4
QUARTERS_PER_YEAR = 4

def _extract_sprint_number(csv_path: str | Path) -> int:
    """Extrae el primer número de 4 dígitos del nombre del CSV."""
    name = Path(csv_path).name
    m = re.search(r"(\d{4})", name)
    if not m:
        raise ValueError(f"No se pudo extraer un sprint de 4 dígitos del nombre: {name}")
    return int(m.group(1))

def _advance_calendar(year: int, quarter: int, sprint_in_quarter: int, delta: int) -> Tuple[int, int, int]:
    """
    Avanza (o retrocede) delta sprints, asumiendo 4 sprints por trimestre y 4 trimestres por año.
    sprint_in_quarter está en [1..4], quarter en [1..4]
    """
    if quarter not in (1, 2, 3, 4):
        raise ValueError(f"Trimestre inválido: {quarter}")
    if sprint_in_quarter not in (1, 2, 3, 4):
        raise ValueError(f"Sprint inválido dentro del trimestre: {sprint_in_quarter}")

    base_index = (quarter - 1) * SPRINTS_PER_QUARTER + (sprint_in_quarter - 1)
    new_index = base_index + delta

    year_delta, idx_in_year = divmod(new_index, QUARTERS_PER_YEAR * SPRINTS_PER_QUARTER)
    new_year = year + year_delta

    new_quarter = (idx_in_year // SPRINTS_PER_QUARTER) + 1
    new_sprint = (idx_in_year % SPRINTS_PER_QUARTER) + 1

    return int(new_year), int(new_quarter), int(new_sprint)

def _resolve_year_quarter_sprint(sprint_num: int) -> Tuple[int, int, int]:
    """
    Devuelve (year, quarter, sprint_in_quarter).
    - Primero mira overrides (casos conocidos).
    - Si no está, calcula por desplazamiento desde BASE_SPRINT_NUM.
    """
    if sprint_num in SPRINT_OVERRIDES:
        return SPRINT_OVERRIDES[sprint_num]

    delta = sprint_num - BASE_SPRINT_NUM
    return _advance_calendar(BASE_YEAR, BASE_QUARTER, BASE_SPRINT_IN_QUARTER, delta)

def _build_out_dir(base_out_dir: str | Path, csv_path: str | Path) -> Path:
    sprint_num = _extract_sprint_number(csv_path)
    year, quarter, sprint_n = _resolve_year_quarter_sprint(sprint_num)
    return Path(base_out_dir) / str(year) / f"Trimestre_{quarter}" / f"Sprint_{sprint_n}"


## 3) Ejecuta el análisis
Puedes ajustar `day_hours` (horas por día) y `BASE_OUT_DIR` (carpeta raíz de salida).

In [ ]:
from IPython.display import display

# Ajustes
day_hours = 8.0
BASE_OUT_DIR = Path("outputs")  # en Colab, se crea en el directorio de trabajo

out_dir = _build_out_dir(BASE_OUT_DIR, csv_path)
print("Out dir:", out_dir)

analyzer = EstimacionesAnalyzer(csv_path=csv_path, day_hours=day_hours).run_all(out_dir=out_dir)

# Vista rápida en pantalla (lo mismo que la tabla HTML)
df_show = analyzer.display_df()
display(df_show)


## 4) Ver y descargar los resultados
Generamos un ZIP con los ficheros y lo descargamos.

In [ ]:
import shutil
from google.colab import files
from pathlib import Path

# out_dir ya es algo como: outputs/2025/Trimestre_4/Sprint_3
# BASE_OUT_DIR es: outputs
rel = out_dir.relative_to(BASE_OUT_DIR)  # 2025/Trimestre_4/Sprint_3

# Crear "Resultados.zip" (sin extensión para make_archive)
zip_base = out_dir.parent / "Resultados"  # p.ej. outputs/2025/Trimestre_4/Resultados
zip_path = Path(shutil.make_archive(
    base_name=str(zip_base),
    format="zip",
    root_dir=str(BASE_OUT_DIR),
    base_dir=str(rel)
))

print("ZIP creado en:", zip_path)
files.download(str(zip_path))
